In [1]:
import mlflow
# for codespace
# mlflow.set_tracking_uri("http://localhost:5000")
# mlflow.set_experiment("test")

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("spyder-experiment")
    

<Experiment: artifact_location='file:///d:/projects/MLops-datatalks-project/src/mlruns/1', creation_time=1716706688548, experiment_id='1', last_update_time=1716706688548, lifecycle_stage='active', name='spyder-experiment', tags={}>

In [2]:
import pandas as pd

df = pd.read_parquet('homeworks/hw1/yellow_tripdata_2023-01.parquet', engine='pyarrow')


df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df['duration'].dt.total_seconds() / 60

df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

In [3]:
from sklearn.feature_extraction import DictVectorizer

# Convert the dataframe into a list of dictionaries
train_dicts = df_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()
dv.fit(train_dicts)

# Get the feature matrix
X_train = dv.transform(train_dicts)

# Get the target vector
y_train = df_filtered.duration.values

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [5]:
# load the test data

categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

df_val = read_data('homeworks/hw1/yellow_tripdata_2023-02.parquet')

val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts) 

y_pred = lr.predict(X_val)

y_val = df_val.duration.values

In [6]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [7]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [8]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [9]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

d:\projects\MLops-datatalks-project\mlflow-env\lib\site-packages\xgboost\core.py:160: UserWarning: [02:12:02] WARNING: C:\b\abs_0fh_d4x2ng\croot\xgboost-split_1713973188995\work\cpp_src\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.03263                           
[1]	validation-rmse:5.62254                           
[2]	validation-rmse:5.50931                           
[3]	validation-rmse:5.43702                           
[4]	validation-rmse:5.36216                           
[5]	validation-rmse:5.34036                           
[6]	validation-rmse:5.33125                           
[7]	validation-rmse:5.30264                           
[8]	validation-rmse:5.29844                           
[9]	validation-rmse:5.29599                           
[10]	validation-rmse:5.29350                          
[11]	validation-rmse:5.28936                          
[12]	validation-rmse:5.28762                          
[13]	validation-rmse:5.28219                          
[14]	validation-rmse:5.27735                          
[15]	validation-rmse:5.27657                          
[16]	validation-rmse:5.27560                          
[17]	validation-rmse:5.27255                          
[18]	valid

KeyboardInterrupt: 

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
import pickle

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")